In [51]:
import pandas as pd
import apiclient.discovery
import httplib2
from oauth2client.service_account import ServiceAccountCredentials

In [52]:
# коннектимся и забираем данные из таблиц

creds_file    = '/Users/roman/Documents/MacBuildServer/creds.json'

credentials   = ServiceAccountCredentials.from_json_keyfile_name(creds_file, ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive'])
httpAuth      = credentials.authorize(httplib2.Http())
service       = apiclient.discovery.build('sheets', 'v4', http = httpAuth)

spreadsheetId = '1Ycg7zTxds9DZnDvTrFcyNNKuTUxg6Yy6WF0a8Wc02WQ'

transactions_ = service.spreadsheets().values().get(spreadsheetId=spreadsheetId, range='transactions!A:D').execute()
clients_      = service.spreadsheets().values().get(spreadsheetId=spreadsheetId, range='clients!A:C').execute()
managers_     = service.spreadsheets().values().get(spreadsheetId=spreadsheetId, range='managers!A:C').execute()
leads_        = service.spreadsheets().values().get(spreadsheetId=spreadsheetId, range='leads!A:F').execute()

In [53]:
# собираем фрейм транзакций

transaction_id = []
created_at = [] 
m_real_amount = []
l_client_id = []

for i in transactions_['values'][1:]:
    transaction_id.append(i[0])
    created_at.append(i[1])
    m_real_amount.append(i[2])
    l_client_id.append(i[3])
    
transactions = pd.DataFrame(list(zip(transaction_id, created_at, m_real_amount,l_client_id)),columns=[ 
                                                transactions_['values'][0][0],
                                                transactions_['values'][0][1],
                                                transactions_['values'][0][2],
                                                transactions_['values'][0][3]])

In [54]:
# собираем фрейм клиентов

client_id = []
created_at = [] 
l_manager_id = []

for i in clients_['values'][1:]:
    if len(i) == 3: # обрабатываем истории, где в created_at и l_manager_id отсутствуют значения. 
                    # Пример: clients_['values'][5893]
                    # Убираем таких клиентов, т.к. по ним нет дополнительной инфы
        client_id.append(i[0])
        created_at.append(i[1])
        l_manager_id.append(i[2])
    continue
    
clients = pd.DataFrame(list(zip(client_id, created_at, l_manager_id)),columns=[ 
                                                clients_['values'][0][0],
                                                clients_['values'][0][1],
                                                clients_['values'][0][2]])

In [55]:
# собираем фрейм менеджеров

manager_id = []
d_manager = [] 
d_club = []

for i in managers_['values'][1:]:
        manager_id.append(i[0])
        d_manager.append(i[1])
        d_club.append(i[2])
    
managers = pd.DataFrame(list(zip(manager_id, d_manager, d_club)),columns=[ 
                                                managers_['values'][0][0],
                                                managers_['values'][0][1],
                                                managers_['values'][0][2]])

In [56]:
# собираем фрейм заявок

lead_id = []
created_at = [] 
d_utm_medium = []
d_utm_source = []
l_manager_id = []
l_client_id = []

for i in leads_['values'][1:]:
    lead_id.append(i[0])
    created_at.append(i[1])
    d_utm_medium.append(i[2])
    d_utm_source.append(i[3])
    l_manager_id.append(i[4])
    l_client_id.append(i[5])
    
leads = pd.DataFrame(list(zip(lead_id, created_at, d_utm_medium, d_utm_source, l_manager_id, l_client_id)),columns=[ 
                                                leads_['values'][0][0],
                                                leads_['values'][0][1],
                                                leads_['values'][0][2],
                                                leads_['values'][0][3],
                                                leads_['values'][0][4],
                                                leads_['values'][0][5]])

In [57]:
# поправим источники
leads['d_utm_source'] = leads['d_utm_source'].replace('vk', 'vkontakte')
leads['d_utm_source'] = leads['d_utm_source'].replace('ycard#!/tproduct/225696739-1498486363994', 'ycard')
leads['d_utm_source'] = leads['d_utm_source'].replace('ig', 'instagram')
leads['d_utm_source'] = leads['d_utm_source'].replace('insta', 'instagram')
leads['d_utm_source'] = leads['d_utm_source'].replace('', 'unknown')
leads['d_utm_source'] = leads['d_utm_source'].replace('utm_source', 'unknown')

# поправим форматы
leads['created_at'] = pd.to_datetime(leads['created_at']).dt.normalize()
transactions['created_at'] = pd.to_datetime(transactions['created_at']).dt.normalize() 
transactions['m_real_amount'] = transactions['m_real_amount'].astype(int)

In [58]:
# забираем инфу о менеджерах
leads_ = leads.merge(managers, how='inner', left_on='l_manager_id', right_on='manager_id')

In [59]:
# количество заявок
all_leads = leads_.groupby(['d_club', 'd_manager', 'd_utm_source', 'created_at'])['lead_id'].count().reset_index()

In [60]:
# количество мусорных заявок
garbage_leads = leads_.query('l_client_id == "00000000-0000-0000-0000-000000000000"')\
                      .groupby(['d_club', 'd_manager', 'd_utm_source', 'created_at'])['lead_id'].count().reset_index()

In [61]:
# клиенты у которых одна заявка
only_one_lead = leads_.query('l_client_id != "00000000-0000-0000-0000-000000000000"')\
                      .groupby('l_client_id').agg({'lead_id': 'count', 'created_at': 'min'})\
                      .reset_index()\
                      .query('lead_id == 1')

# клиенты с одной заявкой, у которых оплата в день заявки или позднее
only_one_lead_clean = transactions[['l_client_id', 'created_at']]\
                     .merge(only_one_lead[['l_client_id', 'created_at']], 
                            how='inner', 
                            on='l_client_id', 
                            suffixes=('_transaction', '_lead'))\
                     .query('created_at_transaction >= created_at_lead')

# количество новых заявок
new_leads = leads_[leads_['l_client_id'].isin(only_one_lead_clean['l_client_id'])]\
            .groupby(['d_club', 'd_manager', 'd_utm_source', 'created_at'])['lead_id'].count().reset_index()

In [62]:
# собираем все заяявки вместе
final_leads = all_leads.merge(new_leads, on=['d_club', 'd_manager', 'd_utm_source', 'created_at'], how='left')\
                       .merge(garbage_leads, on=['d_club', 'd_manager', 'd_utm_source', 'created_at'], how='left')\
                       .fillna(0)\
                       .rename(columns = {'lead_id_x': 'all_leads', 'lead_id_y': 'new_leads', 'lead_id': 'garbage_leads'})

In [63]:
# отрежем возвраты
transactions = transactions.query('m_real_amount > 0')

In [64]:
'''
сгруппируемся и найдем первую дату оплаты, при сумиировании денег тут полагаем, 
что более поздние покупки в другую дату в рамках одной транзакции считаются как одна единая покупка
'''

transactions = transactions.groupby(['transaction_id', 'l_client_id'])\
                           .agg({'created_at': 'min', 'm_real_amount': 'sum'})\
                           .reset_index()

In [65]:
# дата первой покупки
first_pay = transactions.groupby('l_client_id')['created_at'].min()\
                        .reset_index().rename(columns={'created_at': 'first_pay'})

In [66]:
# количество покупок
purchase_cnt = transactions.groupby('l_client_id')['transaction_id'].nunique()\
                           .reset_index().rename(columns={'transaction_id': 'purchase_cnt'})

In [67]:
# добавим дату первой покупки 
transactions_fp = transactions.merge(first_pay, on='l_client_id', how='inner')\
                              .merge(purchase_cnt, on='l_client_id', how='inner')

In [68]:
# соединяем транзакции и заявки
tran_lead = transactions_fp.merge(clients, left_on='l_client_id', right_on='client_id', how='inner', suffixes=('_transaction', '_client'))\
            .merge(managers,left_on='l_manager_id', right_on='manager_id', how='inner', suffixes=('_transaction', '_client'))\
            .drop(['client_id', 'manager_id'], axis = 1)\
            .merge(leads, on=['l_client_id', 'l_manager_id'], how='inner')

In [69]:
# считаем дельту между датой заявки и транзакцией
tran_lead['delta'] = (tran_lead['created_at_transaction'] - tran_lead['created_at']).dt.days

In [70]:
# количество покупателей
buyers = tran_lead.query('delta >= 0 & delta < 8')\
        .groupby(['d_club', 'd_manager', 'd_utm_source', 'created_at_transaction'])['l_client_id'].nunique().reset_index()

In [71]:
# количество новых покупателей

new_buyers = tran_lead.query('delta >= 0 & delta < 8 & created_at_transaction == first_pay & purchase_cnt == 1')\
            .groupby(['d_club', 'd_manager', 'd_utm_source', 'created_at_transaction'])['l_client_id'].nunique().reset_index()

'''
условия delta >= 0 & delta < 8 & created_at_transaction == first_pay отлавливает случаи, когда транзакции 
клиента соответствуют условиям: "купил в течение недели после заявки" и "не покупал раньше". 

Однако есть случаи, когда эти условия соблюдаются и при этом есть более позняя покупка, что делает клиента не новым.
Поэтому добавляем - purchase_cnt == 1.

Чтобы посмотреть примеры, можно убрать условие и поискать на вкладке "transactions" по l_client_id там, где purchase_cnt > 1
tran_lead.query('delta >= 0 & delta < 8 & created_at_transaction == first_pay & purchase_cnt > 1')
'''

'\nусловия delta >= 0 & delta < 8 & created_at_transaction == first_pay отлавливает случаи, когда транзакции \nклиента соответствуют условиям: "купил в течение недели после заявки" и "не покупал раньше". \n\nОднако есть случаи, когда эти условия соблюдаются и при этом есть более позняя покупка, что делает клиента не новым.\nПоэтому добавляем - purchase_cnt == 1.\n\nЧтобы посмотреть примеры, можно убрать условие и поискать на вкладке "transactions" по l_client_id там, где purchase_cnt > 1\ntran_lead.query(\'delta >= 0 & delta < 8 & created_at_transaction == first_pay & purchase_cnt > 1\')\n'

In [72]:

'''
По некоторым клиентам(при соблюдении всех условий) неизбежно дубляться суммы из-за нескольких заявок в один день 
из разных источников(или одного)

36184d79-273e-11eb-ac1d-c412f533dba1
1a32cba7-d5a2-11e9-abbe-c412f533dba1
402cbf93-1db5-11eb-ac1d-c412f533dba1
7a5ed477-21a7-11eb-ac1d-c412f533dba1
9ac01dce-1d34-11eb-ac1d-c412f533dba1
2afc8bc6-1d34-11eb-ac1d-c412f533dba1
2483aa90-177a-11eb-ac1d-c412f533dba1
1da0523f-d5b5-11e9-abbe-c412f533dba1

'''

'\nПо некоторым клиентам(при соблюдении всех условий) неизбежно дубляться суммы из-за нескольких заявок в один день \nиз разных источников(или одного)\n\n36184d79-273e-11eb-ac1d-c412f533dba1\n1a32cba7-d5a2-11e9-abbe-c412f533dba1\n402cbf93-1db5-11eb-ac1d-c412f533dba1\n7a5ed477-21a7-11eb-ac1d-c412f533dba1\n9ac01dce-1d34-11eb-ac1d-c412f533dba1\n2afc8bc6-1d34-11eb-ac1d-c412f533dba1\n2483aa90-177a-11eb-ac1d-c412f533dba1\n1da0523f-d5b5-11e9-abbe-c412f533dba1\n\n'

In [73]:
# телодвижения ниже направлены на исключение раздуплившихся сумм
inc_buyers = tran_lead.query('delta >= 0 & delta < 8 & created_at_transaction == first_pay & purchase_cnt == 1')\
                      .drop(['lead_id', 'created_at', 'd_utm_medium', 'delta'], axis = 1).drop_duplicates()

# жертвуем точностью определения канала привлечения в пользу корректности сумм
inc_buyers = inc_buyers.groupby(['transaction_id', 'created_at_transaction', 'm_real_amount',
       'l_client_id', 'first_pay', 'purchase_cnt', 'created_at_client',
       'l_manager_id', 'd_manager', 'd_club'])['d_utm_source'].min().reset_index()

In [74]:
# доход от покупок новых покупателей
income_buyers = inc_buyers.groupby(['d_club', 'd_manager', 'd_utm_source', 'created_at_transaction'])['m_real_amount'].sum().reset_index()

In [75]:
# собираем все транзакции вместе
final_buyers = buyers.merge(new_buyers, on=['d_club', 'd_manager', 'd_utm_source', 'created_at_transaction'], how='left')\
        .merge(income_buyers, on=['d_club', 'd_manager', 'd_utm_source', 'created_at_transaction'], how='left')\
        .fillna(0)\
        .rename(columns = {'l_client_id_x': 'buyers', 'l_client_id_y': 'new_buyers', 'created_at_transaction': 'created_at'})

In [76]:
# соберем все данные в один фрейм
final = final_leads.merge(final_buyers, on=['d_club', 'd_manager', 'd_utm_source', 'created_at'], how='outer')\
                   .fillna(0)

In [77]:
#поправим формат, который слетел при соединении
final['all_leads'] = final['all_leads'].astype(int)
final['new_leads'] = final['new_leads'].astype(int)
final['garbage_leads'] = final['garbage_leads'].astype(int)
final['garbage_leads'] = final['garbage_leads'].astype(int)
final['buyers'] = final['buyers'].astype(int)
final['new_buyers'] = final['new_buyers'].astype(int)

# поменяем формат у даты, т.к. JSON не принимает datetime
final['created_at'] = final['created_at'].astype(str)

# переименуем заголовки
final.rename(columns={
    'd_club': 'клуб',
    'd_manager': 'менеджер',
    'd_utm_source': 'канал привлечения',
    'created_at': 'дата',
    'all_leads': 'количество заявок',
    'new_leads': 'количество новых заявок',
    'garbage_leads': 'количество мусорных заявок',
    'buyers': 'количество покупателей',
    'new_buyers': 'количество новых покупателей',
    'm_real_amount': 'доход от покупок новых покупателей',
}, inplace=True)

In [78]:
spreadsheet = service.spreadsheets().create(body = {
    'properties': {'title': 'XO', 'locale': 'ru_RU'},
    'sheets': [{'properties': {'sheetType': 'GRID',
                               'sheetId': 0,
                               'title': '1',
                              }}]
}).execute()

In [79]:
# spreadsheet

In [80]:
# доступ к документу
driveService = apiclient.discovery.build('drive', 'v3', http = httpAuth)
shareRes = driveService.permissions().create(
    fileId = spreadsheet['spreadsheetId'],
    body = {'type': 'anyone', 'role': 'writer'},  
    fields = 'id'
).execute()

In [81]:
#записываем данные в гуглодок
results = service.spreadsheets().values().batchUpdate(spreadsheetId = spreadsheet['spreadsheetId'], body = {
    "valueInputOption": "USER_ENTERED",
    "data": [
        {"range": "1!A:J",
         "majorDimension": "ROWS",     # сначала заполнять ряды, затем столбцы (т.е. самые внутренние списки в values - это ряды)
         "values": final.T.reset_index().values.T.tolist()}

       
    ]
}).execute()